In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue

# Croma

In [2]:
class Croma:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.croma.com/')
        self.flag = False

        while self.flag != True:
            try:
                #self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
                self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="searchV2"]')))

                self.un.send_keys(self.search)
                self.flag = True
            except (ElementNotInteractableException, TimeoutException) :
                self.driver.refresh()
                time.sleep(3)
                pass
#                 self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#                 self.un.send_keys(self.search)
#                 self.flag = True
    

    
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
#         try:
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
#         except ElementNotInteractableException:
#             self.driver.refresh()
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
            
        
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review_link = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, self.search)))
        #name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))


        d = {}
        name = []
        for i in review_link:
            name.append(i.text)

        for i, j in  zip(review_link, name):
            x = self.search.lower()
            if x in j.lower() and '(Renewed)' not in j:
                d[i] = j

        time.sleep(1)
        return d
    
    def scrapings(self, d, limit):
        j = 1
        prod_review= {}
        link = {}
        price = {}
        ratings = {}
        avg_rating = {}
        
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            #i.click()
            webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()
            while len(self.driver.window_handles) != 2:
                time.sleep(3)
                webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()

            #driver.execute_script("arguments[0].click();", i)
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            
            y = d[i]
            # scraping link
            l = self.driver.current_url
            link[y] = l
            
            
            # scraping price
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            p = []
            h = x.find_all('span', {'class' : 'amount'})
            
            if len(h) <= 0:
                price[y] = p
            else:
                p.append(h[0].text)
                price[y] = p
            
            
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            #try:
            #    all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'btn-wrap.view-all-review-btn')))
            #all_comments[0].click()
            #   webdriver.ActionChains(driver).move_to_element(all_comments).click(all_comments).perform
            #except TimeoutException:
            #    pass
            
            # scrapping ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            r = []
            rating = {'5 Stars': 0, '4 Stars' : 0, '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}
            l = h = x.find_all('div', {'class' : 'barAndStar'})
            # h = x.find_all('p', {'class' : 'right-text'})
            # rating['4 Stars'] = l[0].text
            if len(l) <= 0:
                ratings[y] = rating
            else:
                count = 5
                for i in l:
                    rating['{} Stars'.format(count)] = i.text[6:]
                    count-=1
                    
                ratings[y] = rating
            
            
            # scraping  avg ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            t = []
            h = x.find_all('div', {'class' : 'totalRating'})
            
            if len(h) <= 0:
                avg_rating[y] = t
            else:
                t.append(h[0].text)
                avg_rating[y] = t

            #time.sleep(3)
            
            
            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 'desc'})
            reviews = set()
            #y = d[i]
            for k in h:
                reviews.add(k.text)
                print(k.text)
                if len(reviews) == self.limit:
                    break
            #driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            #curr_height = driver.execute_script('return document.body.scrollHeight;')
            flag = 0
            reviews1 = reviews
            while len(reviews) != self.limit:
                try:
                    #driver.execute_script('window.scrollBy(0, 2000);')
                    #time.sleep(2)
                    flag+=1
                    more = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'View All Reviews')))
                    webdriver.ActionChains(self.driver).move_to_element(more).click(more).perform()
                    e = self.driver.page_source
                    x= BeautifulSoup(e, 'html.parser')
                    t = x.find_all('div', {'class' : 'desc'})
                    #print(curr_height)
                    #curr_height += 100
                    for l in t:
                        reviews.add(l.text.strip())
                        if len(reviews) == self.limit:
                            break

                    self.driver.execute_script('window.scrollBy(5350, 100);')
                    time.sleep(3)
                    print(flag)
                    if flag == 11:
                        if len(reviews1) == len(reviews):
                            break

                    #driver.execute_script('window.scrollBy(0, 2000);')
                except TimeoutException:
                    flag+=3
                    if flag >= 11 :
                        break
                    else:
                        pass

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review, ratings, link, avg_rating, price
        
        
    def run(self, page_visit):
        links = []
        reviews = []
        all_ratings = []
        prices = []
        avg_r = []
        
        main_d = {}
        now = time.time()
        for i in range(self.page_visit):
            d = self.link_collection()
            print(len(d))
            new_d = {}
            for i, j in d.items():
                if i in main_d:
                    continue
                else:
                    new_d[i] = j
            for i, j in d.items():
                main_d[i] = j
            x, y, z, a, b = self.scrapings(new_d, self.limit)
            reviews.append(x)
            all_ratings.append(y)
            links.append(z)
            avg_r.append(a)
            prices.append(b)
            
            
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(3)
            next_page =  WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn.btn-secondary.btn-viewmore')))
            webdriver.ActionChains(self.driver).move_to_element(next_page).click(next_page).perform()
        
        self.result_queue.put((reviews, prices, avg_r, all_ratings, links))
        
        #return reviews, all_ratings, links, avg_r, prices
#x = Croma('iPhone 14', 10)

    
    

# Amazon

In [3]:
class Amazon:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.result_queue = Queue()
        self.page_visit = page_visit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.amazon.in/')
        self.un = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="twotabsearchtextbox"]')))
        self.un.send_keys(search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        first_page_id = self.driver.current_window_handle
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal'))) # a-size-base.s-underline-text
        reviews = []
        price = []

        for i in review:
            if '₹' not in i.text:
                reviews.append(i)

        for i in review:
            if '₹' in i.text:
                price.append(i.text)
        #price = []

        cnt = 0
        for i in price:
            if cnt%2 != 0:
                price.append(i)
            cnt+=1

        try:
            title1 = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-base-plus.a-color-base.a-text-normal')))
        except TimeoutException:
            title1 = []

        try: 
            title2 = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-medium.a-color-base.a-text-normal')))
        except TimeoutException:
            title2 = []


        main_title = title1 if len(title1) > len(title2) else title2


        titles = [] 
        d = {}
        prices = {}
        for i, j in zip(main_title, reviews):
            x = self.search.lower()
            if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or ('Protect+' not in i.text): # or ('AppleCare' not in i.text) 
                d[j] = i.text
                titles.append(i.text)

        for i, j in zip(main_title, price):
            prices[i.text] = j[1:]


        time.sleep(1)
        return d, titles, prices




    def scraping(self, d, limit, prices):
        # j = 1
        prod_review = {}
        ratings = {}
        link_of_product = {}
        overall_rating = {}
        for i in d:
            y = d[i]
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # driver.switch_to.window(first_page_id)
            i.click()
            time.sleep(2)
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            link = self.driver.current_url
            link_of_product[y] = link

        

            time.sleep(6)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

            try:
                all_comments = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'a-link-emphasis.a-text-bold')))
                all_comments.click()
                time.sleep(2)
            except TimeoutException:
                pass

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            try:
                box = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-dropdown-prompt')))
                for k in box:
                    if k.text == 'All formats':
                        k.click()
            except TimeoutException:
                pass

            try:
                click = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'format-type-dropdown_1')))
                click.click()
                time.sleep(3)
            except TimeoutException:
                pass

             # scrapping views
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            # scraping rating
            h = x.find_all('span', {'class': 'a-size-medium a-color-base'})

            clean = ' '
            for g in h:
                cleaned = g.text.strip()
                if g.text.strip() != '':
                    clean += cleaned[:3]
            overall_rating[y] = cleaned.strip()

            ## scraping review

            t = x.find_all('span', {'class': 'a-size-base review-text review-text-content'})
            reviews = set()
            for l in t:
                reviews.add(l.text.strip())
                if len(reviews) == self.limit:
                    break

            # star ratings
            h = x.find_all('td', {'class': 'a-text-right a-nowrap'})
            rating = {}
            counter = 5
            for rat in h:
                rating['{} Stars'.format(counter)] = rat.text.strip()
                counter -= 1

            ratings[y] = rating


            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    n = 0
                    while len(reviews) != self.limit:
                        try:
                            last = WebDriverWait(self.driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-disabled.a-last')))
                            x = last.text
                            if x == 'Next page':
                                break
                        except (TimeoutException, StaleElementReferenceException):
                            review1 = reviews
                            next_ = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-last')))
                            next_.click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x = BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class': 'a-size-base review-text review-text-content'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == self.limit:
                                    break
                            prod_review[y] = reviews
                            if len(reviews) == len(review1):
                                break
                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])

        return prod_review, ratings, link_of_product, overall_rating

    def run(self, page_visit):
        review_of_product = []
        ratings_of_product = []
        link_product = []
        price_product = []
        overall_r = []

        for i in range(self.page_visit):
            d, titles, prices = self.link_collection()
            print(titles)
            prod, ratings, links, overall = self.scraping(d, self.limit, prices)

            review_of_product.append(prod)
            ratings_of_product.append(ratings)
            link_product.append(links)
            price_product.append(prices)
            overall_r.append(overall)


            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(2)
            flag = False
            while flag != True:
                try:
                    next_ = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                                                's-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
                    next_.click()
                    flag = True
                except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                    self.driver.refresh()
                    time.sleep(3)
                    pass
                
        self.result_queue.put((review_of_product,  price_product, overall_r, ratings_of_product, link_product))
        #return review_of_product, ratings_of_product, link_product, price_product, overall_r



# Flipkart

In [4]:
class Flipkart:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.flipkart.com/')
        
        x_button = WebDriverWait(self.driver, 17).until(EC.presence_of_element_located((By.CLASS_NAME, '_2KpZ6l._2doB4z'))) 
        x_button.click()
        self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')))
        self.un.send_keys(self.search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
        
    
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_1fQZEK')))
        title = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))

        titles = [] 
        d = {}
        reviews = []

        try:
            for i in review:
                if '(' in i.text:
                    reviews.append(i)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()
        try:
            for i, j in zip(title, reviews):
                #x = search.lower()
                if '(Renewed)' not in i.text:
                    d[j] = i.text
                    titles.append(i.text)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()

        if len(d) == 0:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()



        time.sleep(1)
        return d
    
    
    def scraping(self, d, limit):
        j = 1
        prod_review= {}
        link = {}
        overall_rating = {}
        price = {}
        ratings = {}
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            i.click()
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            
            
            y = d[i]
        
            # scraping link
            url = self.driver.current_url
            link[y] = url
            
            
            # scraping stars
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_1uJVNT'})
            stars = {}
            count = 5
            for i in h:
                stars["{} Stars".format(count)] = i.text
                count-=1

            ratings[y] = stars


            #scraping price   <div class="_30jeq3 _16Jk6d">₹69,999</div>
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_30jeq3 _16Jk6d'})
            p = []
            for i in h:
                p.append(i.text)
            price[y] = p


            # average ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_2d4LTz'})
            r = []
            for i in h:
                r.append(i.text)
            overall_rating[y] = r
            
            
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            all_comments_link = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, '_3UAT2v._16PBlm'))) 
            self.driver.execute_script("arguments[0].click();", all_comments_link)

            #time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 't-ZTKy'})
            reviews = set()
           # y = d[i]
            for k in h:
                reviews.add(k.text)
                if len(reviews) == limit:
                    break


            if len(reviews) != self.limit or len(reviews) < self.limit:
                    try:
                        while len(reviews) != self.limit:
                            review1 = reviews
                            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                            if len(next_page) == 1:
                                next_page[0].click()
                            elif len(next_page) == 2:
                                next_page[1].click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x= BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class' : 't-ZTKy'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == limit:
                                    break
                            prod_review[y] = reviews

                    except TimeoutException:
                        pass
            else:
                prod_review[y] = reviews

            #prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review, link, overall_rating, price, ratings

    
    def run(self, page_visit):
        review = []
        avg_rating = []
        prices = []
        links = []
        all_r = []
        for i in range(self.page_visit):
            d = self.link_collection()
            print(len(d))
            x, y, z , a, b = self.scraping(d, self.limit)
            review.append(x)
            links.append(y)
            avg_rating.append(z)
            prices.append(a)
            all_r.append(b)
            
            #driver.switch_to.window(driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
            try:
                if len(next_page) == 1:
                    next_page[0].click()
                elif len(next_page) == 2:
                    next_page[1].click()
            except ElementClickInterceptedException:
                self.driver.refresh()
                time.sleep(4)
                self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                
                flag = False
                while flag != True:
                    try:
                        next_page = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                        if len(next_page) == 1:
                            next_page[0].click()
                        elif len(next_page) == 2:
                            next_page[1].click()
                        flag = True
                    except (ElementNotInteractableException, TimeoutException, ElementClickInterceptedException):
                        self.driver.refresh()
                        time.sleep(3)
                        pass
                    
        self.result_queue.put((review, prices, avg_rating, all_r, links))

        #return review, avg_rating, prices, links, all_r

In [5]:
# call

x = Croma('iPhone 14', 30, 1)
y = Flipkart('iPhone 14', 30, 1)
z = Amazon('iPhone 14', 30, 1)

t1 = threading.Thread(target=x.run(1))
t2 = threading.Thread(target=y.run(1))
t3 = threading.Thread(target=z.run(1))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

20


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E7DCE3+50899]
	(No symbol) [0x00E0E111]
	(No symbol) [0x00D15588]
	(No symbol) [0x00CFD333]
	(No symbol) [0x00D5F4DB]
	(No symbol) [0x00D6DB33]
	(No symbol) [0x00D5B6F6]
	(No symbol) [0x00D37708]
	(No symbol) [0x00D3886D]
	GetHandleVerifier [0x010E3EAE+2566302]
	GetHandleVerifier [0x011192B1+2784417]
	GetHandleVerifier [0x0111327C+2759788]
	GetHandleVerifier [0x00F15740+672048]
	(No symbol) [0x00E18872]
	(No symbol) [0x00E141C8]
	(No symbol) [0x00E142AB]
	(No symbol) [0x00E071B7]
	BaseThreadInitThunk [0x7716FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B27B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77B27B2E+238]


In [20]:
# data collection
review = []
price = []
avg_rating = []
all_rating = []
link = []

for i in [x, y, z]:
    x, y, z, a, b = i.result_queue.get()
    review.append(x)
    price.append(y)
    avg_rating.append(z)
    all_rating.append(a)
    link.append(b)

In [21]:

import re
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


count = 0
length = 0

class Cleaning:
    def __init__(self, review, price, avg_rating, all_rating, link):
        self.review = review
        self.price = price
        self.avg_rating = avg_rating
        self.all_rating = all_rating
        self.link = link
        
        self.df = pd.DataFrame(
            {'Product Name': [], 'Review': [], '5 Stars': [], '4 Stars': [], '3 Stars': [], '2 Stars': [],
             '1 Stars': [], 'Avg Rating': [], 'Price': [], 'Link': []})

    def clean(self, review, price, avg_rating, all_rating, link):
        count = 0
        length1 = 0
        
        for i in self.review:
            length = 0
            for j in i:
                #length = 0
                for k, l in j.items():
                    #print(k)
                    for m in l:
                        self.df.loc[count] = [k, m, self.all_rating[length1][length][k]['5 Stars'], 
                                              self.all_rating[length1][length][k]['4 Stars'], 
                                              self.all_rating[length1][length][k]['3 Stars'],
                                              self.all_rating[length1][length][k]['2 Stars'],
                                              self.all_rating[length1][length][k]['1 Stars'],
                                              self.avg_rating[length1][length][k],
                                              self.price[length1][length][k], self.link[length1][length][k] ]
                        count += 1
                length += 1
            length1 +=1
        
        
        # cleaning rating 
        x = self.df['Avg Rating']
        index = 0
        for i in x:
            if type(i) == list:
                y = i[0].strip()
                print(float(y))
                self.df.loc[index, 'Avg Rating'] = y
            else:
                if type(i) == str:
                    string = ' '
                    for j in i:
                        if  j.isalpha():
                            continue
                        string+= j
                    string = string.strip()
                    y = float(string[:4])
                    print(y)
                    self.df.loc[index, 'Avg Rating'] = y
            index+=1
            
        
        # cleeaning price
        x = self.df['Price']
        index  = 0
        for i in x:
            if type(i) == list:
                y = i[0]
                y = y.split('.')
                y =y[0]
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string+=j
                y = float(string)
                print(y)
                self.df.loc[index, 'Price'] = y
            else:
                y = i
                if '\n' in y:
                    y = y.split('\n')
                    y = y[0]
                y = y.split('.')
                y = y[0]
                y = re.sub(r'[\n₹,]', '', y)
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string+=j
                if len(string) == 0 or len(string) == 1:
                    string = 0
                else:
                    y = float(string)
                print(y)
                self.df.loc[index, 'Price'] = y

            index += 1
            
         # cleaning all stars
        for i in range(len(self.df)):
            count = 5
            total = 0
            while count != 0:
                x = self.df['{} Stars'.format(count)][i]
                x = str(x)
                x = x.strip()
                x = re.sub(r'[,\n%]', '', x)
                x = int(x)
                total+=x
                self.df.loc[i, '{} Stars'.format(count)] = x
                count-=1
            count1 = 5
            while count1!= 0:
                x = self.df['{} Stars'.format(count1)][i]
                percent = (x / total) * 100
                self.df.loc[i, '{} Stars'.format(count1)] = percent
                count1 -= 1
                
        # adding website name 
        for i, j in enumerate(self.df.Link):
            x  = re.sub(r'https://www.', '', j)
            try:
                x = x[:x.index('.com')] 
            except ValueError:
                x = x[:x.index('.in')] 
            self.df.loc[i, "Website"] = x
        
        
        # clean review
        cleaned = []
        for i in self.df['Review']:
            x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
            y = re.sub(r'READ MORE', '', x)
            # y = re.sub(r'[^\x00-\x7F]+', '', x)
            cleaned.append(y)

        token = []
        stop_words = set(stopwords.words('english'))
        for i in cleaned:
            word_tokens = word_tokenize(i)
            filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
            word = ' '.join(filtered_words)
            words = re.sub('[^a-zA-Z0-9 \n]', '', word)

            print(words)
            token.append(words)
        self.df['Cleaned'] = token

        return self.df

In [22]:
x = Cleaning(review, price, avg_rating, all_rating, link)
df = x.clean(review, price, avg_rating, all_rating, link)
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:883: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.5
4.5
4.5
4.5
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8


81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
71999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0
81999.0


Best smartphone price range overall great performance  great camera great battery
Thanks Flipkart service fine nd comes daysproduct genuine nd love
first IOS device like  gifted younger brother loved  iPhone inhand feel damn premium also camera quality top notch honest bought phone smooth processor camera quality  go
camera expected evening pictures dull brightso much dissipointed compare camera even sfe one plus nord premium phonesat bright sunlight bright light night pictures good portrait modeselfies low light terrible  looks like camera k phonebattery good last day easilyperformance gooddisplay may hz good price range 
Iphone Awesome  Easy carry phone  Light weight  effective daily usage  Battery effective according daily usage  Pictures  sound output  always bestCamera always amazing cinematic video wow  Colour awesome overall good
product genuinely awesome  ca nt stop etes looking product  Awesome work APPLE good customer support seller Flipkart  THANKS PRODUCT  GO
iPhone starlig

power packed product considering price  comes battery  inches larger display base model  Camera superb terms video recordingcinematic mode normal mode K format  feels like iphone pro max hands front  revolutionary product 
phone really good price point  higher refresh rate though would highly appreciated  build quality great  hopefully would get used notch  Best part definitely software  iOS really smooth observe couple minor bugs  OS reliable overall 
Shifted Android Ios first time  observations months  Immediately noticed buttery smooth experience  Everything quick ios  App opening  app loading  changing settings etc  Camera performance excellent  Liked portrait mode much  Battery perfomance excellent well  getting hrs SOT even charging fully  Usually unplugs around  shortcuts app  create various shortcuts automating tasks 
Great Phone price range  hz refresh rate feels smooth  great camera  big screen  good battery backup superb gaming  else anyone ask  Highly Recommended 
Amazing p

,Product Name,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link,Website,Cleaned
0,"Apple iPhone 14 (128GB, Blue)",Nice experience,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,croma,Nice experience
1,"Apple iPhone 14 (128GB, Blue)",Great product and service.,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,croma,Great product service
2,"Apple iPhone 14 (128GB, Blue)",It was great.,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,croma,great
3,"Apple iPhone 14 (128GB, Blue)",Nice price and good service,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,croma,Nice price good service
4,"Apple iPhone 14 (128GB, Blue)",Excellent,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,71990.0,https://www.croma.com/apple-iphone-14-128gb-bl...,croma,Excellent
...,...,...,...,...,...,...,...,...,...,...,...,...
1117,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Video Player is loading.Play VideoPlayMuteCurr...,78.0,9.0,3.0,1.0,9.0,4.5,149900.0,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,amazon,Video Player loadingPlay VideoPlayMuteCurrent ...
1118,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Product is used cause I have been using only A...,78.0,9.0,3.0,1.0,9.0,4.5,149900.0,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,amazon,Product used cause using Apple phones since la...
1119,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,The price is on higher side but the iPhone com...,78.0,9.0,3.0,1.0,9.0,4.5,149900.0,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,amazon,price higher side iPhone comes best performanc...
1120,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,The media could not be loaded.\n ...,78.0,9.0,3.0,1.0,9.0,4.5,149900.0,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,amazon,media could loaded Preordered th September p...


84.31372549019608